# 变分自动编码器

## 自动编码器 Autoencoder

是一类 **无监督神经网络模型**，它的目标是让输入经过网络压缩后，再尽可能准确地重建出来。它本质上是一种 **数据压缩 + 特征学习** 的方法。

- 将高维输入数据压缩成低维表征向量的编码器神经网络
- 通过解压将给定的表征向量还原到原始域的解码器神经网络


神经网络会在训练的过程中寻找编码器和解码器的权重，将原始输入与输出重建之间的损失降到最低

## 自动编码器的组成部分

典型自动编码器由三部分组成：

### **1. 编码器（Encoder）**

* 将输入数据压缩到一个低维空间
* 输出通常叫做 **latent vector / bottleneck / embedding**
* 实际由若干卷积层+激活函数堆叠而成，展平后全连接输入潜在空间

### **2. 潜在空间（Latent Space）**

* 保留输入的关键信息
* 维度比原始数据小；但是为了表示更加细节的照片通常维度上百
* 是自动编码器学习到的“精华特征表示”

### **3. 解码器（Decoder）**

* 将 latent vector 解压回原数据空间
* 目标是尽量重建出原始输入
* 具体可以由若干卷积置换（每一像素周围0填充）+ 激活函数 还原至原始域
* 当然也可以考虑使用上采样（每一行/列像素复制）


## 损失函数
常用均方根误差RMSE或二元交叉熵；注意后者会在预测出现错误的情况施加很重的惩罚，因此这种损失函数由导致像素预测都靠近中间地带的倾向。生成对图像缺乏活力

## 自动编码器的缺点
- latent space中点并不是以（0，0）为中心对称，也没有边界。例如，y轴负方向上的点可能远多余正方向上的点。这导致我们随机采样得到的点可能并不具备实际意义。
> 然而我们希望在隐空间中随机选择一个点，解码器就能给我们生成看上去很真实的图像。
- 也有些数字的表示位于非常小的区域，而有些数字的表示则占用了很大的区域。这使生成的图像缺乏多样性
> 理想情况下，从隐空间中随机采样时应该会获得大致相同的数字分布。但在当前这个空间中随机选择点时，生成某一图像的概率可能一直很高
- 各个颜色之间存在很大间隙，这些间隙上几乎没有任何点
> 这导致生成的图片质量不佳

# 变分自动编码器
改进了损失函数和编码器

### 编码器
- 自动编码器中每个图像都直接映射成隐空间中的一个点、
- 变分自编码器中每个图象都映射成隐空间中围绕某个点的多元正态分布

## 正态分布（Normal distribution / Gaussian）

### 一维正态分布
若随机变量 (X) 的概率密度函数（pdf）为

$f_X(x)=\frac{1}{\sqrt{2\pi},\sigma}\exp!\Big(-\frac{(x-\mu)^2}{2\sigma^2}\Big),\qquad x\in\mathbb{R},$

则称 (X) 服从参数为 (\mu)（均值）和 (\sigma^2)（方差）的正态分布，记作
$
X\sim\mathcal{N}(\mu,\sigma^2).
$

* (\mu\in\mathbb{R})：位置参数（期望值）。
* (\sigma>0)：尺度参数（标准差），方差为 (\sigma^2)。

---

### 累积分布函数（CDF）

正态分布的累积分布函数没有初等初等函数的解析形式，但与误差函数（erf）相关：
$
F_X(x)=\Pr(X\le x)=\frac{1}{2}\Big[1+\operatorname{erf}!\Big(\frac{x-\mu}{\sigma\sqrt{2}}\Big)\Big],
$
其中
$(\operatorname{erf}(z)=\tfrac{2}{\sqrt{\pi}}\int_0^z e^{-t^2},dt)$.

---

### 标准正态分布和标准化

标准正态分布是 (\mu=0,\ \sigma^2=1) 的情况：
$
Z\sim\mathcal{N}(0,1),\qquad \phi(z)=\frac{1}{\sqrt{2\pi}}e^{-z^2/2}.
$
任意 $(X\sim\mathcal{N}(\mu,\sigma^2))$ 可标准化为
$
Z=\frac{X-\mu}{\sigma}\sim\mathcal{N}(0,1).
$

---

### 常用性质（矩与统计量）

* 期望： $(\mathbb{E}[X]=\mu.)$
* 方差： $(\operatorname{Var}(X)=\sigma^2.)$
* 所有阶矩存在；中心矩： $(\mathbb{E}[(X-\mu)^{2k+1}]=0)$，偶数阶中心矩可用双阶乘表示。
* 特征函数（characteristic function）：
  $
  \varphi_X(t)=\mathbb{E}[e^{itX}]=\exp!\big(it\mu-\tfrac{1}{2}\sigma^2 t^2\big).
  $
* 动差母函数（MGF，若存在）：
  $
  M_X(t)=\mathbb{E}[e^{tX}]=\exp!\big(t\mu+\tfrac{1}{2}\sigma^2 t^2\big).
  $

---

### 密度归一化常数的推导（简要）

需要证明
$(\int_{-\infty}^{\infty} e^{-x^2/(2\sigma^2)},dx=\sqrt{2\pi}\sigma)$。
常见方法：令 $(I=\int_{-\infty}^\infty e^{-a x^2}dx)$，平方并用极坐标可得 $(I=\sqrt{\pi/a})$。取 $(a=\tfrac{1}{2\sigma^2})$ 即得所需结果，从而得到 pdf 前的常数 $(\tfrac{1}{\sqrt{2\pi}\sigma})$。

---

### 中心极限定理（Central Limit Theorem, CLT）

若 $(X_1,\dots,X_n)$ 为独立同分布$（i.i.d.）$且具有有限均值$ (\mu) $与方差$ (\sigma^2)$，则标准化样本和
$
\frac{\sum_{i=1}^n X_i - n\mu}{\sigma\sqrt{n}}
$
在 $(n\to\infty)$ 时分布趋近于标准正态 $(\mathcal{N}(0,1))$。这说明为何正态分布在实践中极为普遍。

---

### 参数估计（MLE）

给定样本$ (x_1,\dots,x_n)$（假设独立来自 $(\mathcal{N}(\mu,\sigma^2))$），最大似然估计（MLE）为：

* 均值估计（MLE）：
  $
  \hat\mu=\bar{x}=\frac{1}{n}\sum_{i=1}^n x_i.
  $
* 方差的 MLE（有偏）：
  $
  \hat\sigma^2_{\mathrm{MLE}}=\frac{1}{n}\sum_{i=1}^n (x_i-\bar{x})^2.
  $
* 无偏样本方差通常使用 (1/(n-1))：
  $
  s^2=\frac{1}{n-1}\sum_{i=1}^n (x_i-\bar{x})^2.
  $

---

### 置信区间与假设检验（常用公式）

当样本来自正态总体且 (\sigma^2) 已知时，(\bar X) 的 (100(1-\alpha)%) 置信区间为
$
\bar X \pm z_{1-\alpha/2},\frac{\sigma}{\sqrt{n}},
$
其中 $(z_{1-\alpha/2})$ 是标准正态上相应分位数。当 $(\sigma) $未知且样本来自正态分布时，用 (t) 分布替代。

---

### 多元正态分布（向量形式）

若随机向量 (\mathbf{X}\in\mathbb{R}^d) 的概率密度为
$
f(\mathbf{x})=\frac{1}{(2\pi)^{d/2}|\Sigma|^{1/2}}\exp!\Big(-\tfrac{1}{2}(\mathbf{x}-\boldsymbol{\mu})^\top\Sigma^{-1}(\mathbf{x}-\boldsymbol{\mu})\Big),
$
则称 $(\mathbf{X}\sim\mathcal{N}(\boldsymbol{\mu},\Sigma))$，其中 $(\boldsymbol{\mu})$ 是均值向量，(\Sigma) 是正定协方差矩阵。常用性质：

* 线性仿射变换保正态性：若 $(\mathbf{Y}=A\mathbf{X}+b)$，则 $(\mathbf{Y})$ 仍为正态，均值与协方差为相应线性变换。
* 条件分布与边缘分布仍为正态，且条件均值与条件协方差有闭式表达式（高斯的便利性之一）。

---

### 条件分布（一维情形的直观）

若 $(\begin{pmatrix}X\Y\end{pmatrix})$ 联合服从二维正态，则条件分布 $(X|Y=y)$ 仍为正态，其均值与方差可用协方差表示（写出公式往往在多元部分看到）。

---

### 熵（信息论视角）

对一维正态 $(X\sim\mathcal{N}(\mu,\sigma^2))$，微分熵为
$
h(X)=\frac{1}{2}\log(2\pi e \sigma^2).
$
正态在给定方差下具有最大熵（即“最不确定”）的连续分布。

---

### 与误差函数的关系（CDF 的替代写法）

标准正态的 CDF $(\Phi(z))$ 与误差函数关系：
$
\Phi(z)=\frac{1}{2}\Big[1+\operatorname{erf}\big(\tfrac{z}{\sqrt{2}}\big)\Big].
$
因此计算概率常用数值表或计算器/软件函数（如 (\Phi) 或 erf）。

---

### 常见数值示例

* 大约 (68.27%) 的质量在 $(\mu\pm1\sigma)$ 内；
* (95.45%) 在 $(\mu\pm2\sigma)$ 内（更精确常写为 $(1.96\sigma)$ 对应 (95%)）；
* (99.73%) 在 $(\mu\pm3\sigma)$ 内。

---

### 为什么正态重要（直观总结）

* 中心极限定理使其在很多独立小影响累积的情况下天然出现。
* 数学性质优良（闭式特征函数、线性变换不改变正态性、条件与边缘仍正态等），方便推导和建模。
* 在统计推断（t检验、线性回归等）与信号处理、物理学、金融等领域广泛应用。



### **多维正态分布**
- 正态分布可以从一维推广到 $k$ 维。
- 概率密度函数的公式为：

$
f(x_1, ..., x_k) = \frac{\exp\left(-\frac{1}{2}(x - \mu)^T \Sigma^{-1}(x - \mu)\right)}{\sqrt{(2\pi)^k |\Sigma|}}
$

其中：
- $x$ 是一个$k$ 维向量。
- $\mu$ 是均值向量。
- $\Sigma$ 是协方差矩阵，表示各维度之间的相关性。
- $|\Sigma|$ 是协方差矩阵的行列式。

---

### 2. **二维情况举例**
- 在二维情况下：
  - 均值向量 $\mu = \begin{pmatrix} \mu_1 \\ \mu_2 \end{pmatrix}$
  - 协方差矩阵 $\Sigma = \begin{pmatrix} \sigma_1^2 & \rho \sigma_1 \sigma_2 \\ \rho \sigma_1 \sigma_2 & \sigma_2^2 \end{pmatrix}$
- 其中 $\rho$是 $x_1$ 和 $x_2$ 之间的**相关系数**。

---

### 3. **变分自动编码器（VAE）中的假设**
- VAE 假设**隐空间（latent space）的各个维度之间没有相关性**。
- 因此，协方差矩阵 $\Sigma$ 是一个**对角矩阵**，非对角线上的元素（协方差）都为 0。
- 这意味着编码器只需为每个输入输出：
  - 一个**均值向量 $\mu$ **
  - 一个**方差向量**（实际输出的是**方差的对数 $\log(\text{var})\)**）

---

### 4. **为什么使用对数方差（log_var）**
- 方差必须是正数，但对数方差可以是任意实数（$\infty$ 到 $ +\infty\ $）。
- 这与神经网络输出的自然范围一致，便于训练。

---

### 5. **从分布中采样得到隐向量 \(z\)**
- 编码器输出 $ \mu $ 和 $ \log(\text{var})$ 。
- 若要得到隐空间中的一个具体点 \(z\)，使用以下公式采样：

$
z = \mu + \sigma \times \epsilon
$

其中：
- $\sigma = \sqrt{\text{var}}$，通过对数方差还原得到。
- $\epsilon$ 是从标准正态分布 $N(0, 1)$ 中采样的随机噪声。



## Lambda 层

### 基本概念
- **作用**：将任意函数包装成Keras层
- **使用场景**：当需要Keras层不直接支持的张量操作时
- **示例**：`Lambda(lambda x: x ** 2)` - 将输入值平方

### 在VAE中的应用
- 用于实现重参数化技巧（reparameterization trick）
- 连接编码器和解码器

---

## 损失函数

### VAE损失函数的两个组成部分

#### 1. 重建损失（Reconstruction Loss）
- **作用**：衡量原始图像与重建图像之间的差异
- **计算方法**：均方误差（MSE）
- **代码示例**：
  ```python
  r_loss = K.mean(K.square(y_true - y_pred), axis=[1,2,3])
  ```

#### 2. KL散度损失（KL Divergence Loss）

##### 数学定义
- **解析解公式**：
  $ kl\_loss = -0.5 \times \sum(1 + log\_var - mu^2 - exp(log\_var)) $
- **数学符号表示**：
  $ D_{KL}[N(\mu, \sigma) \| N(0,1)] = \frac{1}{2} \sum(1 + log(\sigma^2) - \mu^2 - \sigma^2) $

##### 物理意义
- 衡量编码器输出的分布与标准正态分布之间的差异
- **最小值条件**：当所有维度满足：
  - `mu = 0` 
  - `log_var = 0`（即方差=1）
- 随着mu和log_var偏离0，KL散度损失增加

##### 直观理解（比喻）
- **mu ≠ 0**：如同"Epsilon不得不将梯子从墙中央挪开"
- **log_var ≠ 0**：如同"N.Coder先生对标记位置的信心不足"
- 两者都会导致"额外的成本"

---

## 损失函数平衡策略

### 总损失函数
```
总损失 = 重建损失 × r_loss_factor + KL散度损失
```

### 权重调整的重要性
1. **重建损失权重过大**：
   - KL散度正则化效果不足
   - 出现普通自动编码器的问题（过拟合）

2. **重建损失权重过小**：
   - KL散度损失占主导
   - 重建图像质量差

3. **平衡策略**：
   - `r_loss_factor`是需要调优的超参数
   - 确保两项损失保持适当平衡

---

## 关键优势

1. **隐空间规整化**：
   - 强制编码分布接近标准正态分布
   - 便于从分布中采样生成新样本

2. **空间利用优化**：
   - 防止点簇之间形成大的间隙
   - 编码器对称地利用原点周围空间

这些要点完整描述了VAE中Lambda层的使用方法和损失函数的设计原理，包括如何平衡重建质量与隐空间规整化。